# PyTorch: The Quiet Revolution in Python Numerical Computing

**Why PyTorch isn't just for ML—it's the future of accelerated math in Python**

---

NumPy has been the backbone of numerical Python for decades. But there's a shift happening that few are talking about: **PyTorch is becoming a superior general-purpose numerical computing library**—not just for machine learning.

### The key insight:
When you write your numerical code in PyTorch, you get:
1. **GPU acceleration for free** — move your tensors to GPU with `.to('cuda')` and your existing code runs on accelerators
2. **Automatic differentiation** — gradients without manual derivation
3. **Future-proof compatibility** — as PyTorch improves (better kernels, new hardware support), your code gets faster without changes
4. **ML-ready data** — your computations are already in tensors, ready for model integration

This notebook benchmarks PyTorch against NumPy across common numerical operations to demonstrate the performance characteristics and the "write once, accelerate anywhere" paradigm.

## Setup & Configuration

In [1]:
import numpy as np
import torch
import time
import pandas as pd
from typing import Callable, Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Check available devices
print("="*60)
print("SYSTEM CONFIGURATION")
print("="*60)
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"MPS available (Apple Silicon): {torch.backends.mps.is_available()}")
print("="*60)

SYSTEM CONFIGURATION
NumPy version: 2.1.2
PyTorch version: 2.8.0+cu129
CUDA available: True
CUDA version: 12.9
GPU: NVIDIA RTX PRO 6000 Blackwell Workstation Edition
GPU Memory: 101.9 GB
MPS available (Apple Silicon): False


In [2]:
# Device configuration
CPU = 'cpu'
GPU = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else None

DEVICES = [CPU]
if GPU:
    DEVICES.append(GPU)
    
print(f"Benchmarking on devices: {DEVICES}")

Benchmarking on devices: ['cpu', 'cuda']


## Benchmarking Infrastructure

In [3]:
def benchmark(func: Callable, n_warmup: int = 3, n_runs: int = 10, sync_cuda: bool = True) -> Dict:
    """
    Benchmark a function with proper warmup and CUDA synchronization.
    
    Args:
        func: Function to benchmark (should return None or a tensor)
        n_warmup: Number of warmup iterations
        n_runs: Number of timed iterations
        sync_cuda: Whether to synchronize CUDA before timing
    
    Returns:
        Dict with timing statistics
    """
    # Warmup
    for _ in range(n_warmup):
        func()
    
    # Synchronize before timing
    if sync_cuda and torch.cuda.is_available():
        torch.cuda.synchronize()
    
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        func()
        
        # Synchronize after each run for accurate GPU timing
        if sync_cuda and torch.cuda.is_available():
            torch.cuda.synchronize()
            
        times.append(time.perf_counter() - start)
    
    return {
        'mean': np.mean(times) * 1000,  # Convert to ms
        'std': np.std(times) * 1000,
        'min': np.min(times) * 1000,
        'max': np.max(times) * 1000,
    }


def format_speedup(baseline: float, comparison: float) -> str:
    """Format speedup as a string with color indication."""
    if comparison < baseline:
        speedup = baseline / comparison
        return f"🚀 {speedup:.1f}x faster"
    else:
        slowdown = comparison / baseline
        return f"🐢 {slowdown:.1f}x slower"

---

# Benchmark 1: Matrix Operations

Matrix multiplication is the bread and butter of numerical computing. Let's see how NumPy compares to PyTorch on CPU and GPU.

In [4]:
def benchmark_matmul(sizes: List[int] = [256, 512, 1024, 2048, 4096]) -> pd.DataFrame:
    """Benchmark matrix multiplication across different sizes and devices."""
    results = []
    
    for size in sizes:
        print(f"\nMatrix size: {size}x{size}")
        
        # NumPy
        a_np = np.random.randn(size, size).astype(np.float32)
        b_np = np.random.randn(size, size).astype(np.float32)
        numpy_time = benchmark(lambda: np.matmul(a_np, b_np))
        print(f"  NumPy:         {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', **numpy_time})
        
        # PyTorch CPU
        a_cpu = torch.from_numpy(a_np)
        b_cpu = torch.from_numpy(b_np)
        torch_cpu_time = benchmark(lambda: torch.matmul(a_cpu, b_cpu))
        print(f"  PyTorch (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', **torch_cpu_time})
        
        # PyTorch GPU (if available)
        if GPU:
            a_gpu = a_cpu.to(GPU)
            b_gpu = b_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: torch.matmul(a_gpu, b_gpu))
            print(f"  PyTorch (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 1: Matrix Multiplication (A @ B)")
print("="*60)
matmul_results = benchmark_matmul()

BENCHMARK 1: Matrix Multiplication (A @ B)

Matrix size: 256x256
  NumPy:         0.11 ± 0.11 ms
  PyTorch (CPU): 0.06 ± 0.00 ms 🚀 1.7x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 7.0x faster

Matrix size: 512x512
  NumPy:         0.32 ± 0.01 ms
  PyTorch (CPU): 2.99 ± 4.69 ms 🐢 9.4x slower
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 15.9x faster

Matrix size: 1024x1024
  NumPy:         2.61 ± 4.82 ms
  PyTorch (CPU): 3.87 ± 4.30 ms 🐢 1.5x slower
  PyTorch (GPU): 0.04 ± 0.00 ms 🚀 59.0x faster

Matrix size: 2048x2048
  NumPy:         5.88 ± 0.40 ms
  PyTorch (CPU): 9.38 ± 0.07 ms 🐢 1.6x slower
  PyTorch (GPU): 0.24 ± 0.02 ms 🚀 24.2x faster

Matrix size: 4096x4096
  NumPy:         46.70 ± 0.16 ms
  PyTorch (CPU): 96.85 ± 9.17 ms 🐢 2.1x slower
  PyTorch (GPU): 1.69 ± 0.00 ms 🚀 27.6x faster


---

# Benchmark 2: Element-wise Operations

Many scientific computations involve element-wise operations on large arrays. These are highly parallelizable—perfect for GPUs.

In [5]:
def benchmark_elementwise(sizes: List[int] = [1_000_000, 10_000_000, 50_000_000, 100_000_000]) -> pd.DataFrame:
    """Benchmark element-wise operations: sin, exp, and compound expressions."""
    results = []
    
    for size in sizes:
        print(f"\nArray size: {size:,} elements")
        
        # Test expression: sin(x) * exp(-x^2) - common in physics/signal processing
        
        # NumPy
        x_np = np.random.randn(size).astype(np.float32)
        numpy_time = benchmark(lambda: np.sin(x_np) * np.exp(-x_np**2))
        print(f"  NumPy:         {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', 'operation': 'sin*exp', **numpy_time})
        
        # PyTorch CPU
        x_cpu = torch.from_numpy(x_np)
        torch_cpu_time = benchmark(lambda: torch.sin(x_cpu) * torch.exp(-x_cpu**2))
        print(f"  PyTorch (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', 'operation': 'sin*exp', **torch_cpu_time})
        
        # PyTorch GPU
        if GPU:
            x_gpu = x_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: torch.sin(x_gpu) * torch.exp(-x_gpu**2))
            print(f"  PyTorch (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', 'operation': 'sin*exp', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 2: Element-wise Operations (sin(x) * exp(-x²))")
print("="*60)
elementwise_results = benchmark_elementwise()

BENCHMARK 2: Element-wise Operations (sin(x) * exp(-x²))

Array size: 1,000,000 elements
  NumPy:         0.84 ± 0.08 ms
  PyTorch (CPU): 0.23 ± 0.01 ms 🚀 3.6x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 39.0x faster

Array size: 10,000,000 elements
  NumPy:         14.87 ± 0.44 ms
  PyTorch (CPU): 14.76 ± 0.37 ms 🚀 1.0x faster
  PyTorch (GPU): 0.13 ± 0.00 ms 🚀 113.0x faster

Array size: 50,000,000 elements
  NumPy:         79.53 ± 0.51 ms
  PyTorch (CPU): 90.68 ± 1.52 ms 🐢 1.1x slower
  PyTorch (GPU): 1.49 ± 0.00 ms 🚀 53.5x faster

Array size: 100,000,000 elements
  NumPy:         157.20 ± 0.34 ms
  PyTorch (CPU): 185.84 ± 1.74 ms 🐢 1.2x slower
  PyTorch (GPU): 2.98 ± 0.00 ms 🚀 52.7x faster


---

# Benchmark 3: Linear Algebra (SVD, Eigendecomposition)

Linear algebra operations are critical for scientific computing, dimensionality reduction, and solving systems of equations.

In [6]:
def benchmark_linalg(sizes: List[int] = [256, 512, 1024, 2048]) -> pd.DataFrame:
    """Benchmark linear algebra operations: SVD and eigendecomposition."""
    results = []
    
    for size in sizes:
        print(f"\n{'='*40}")
        print(f"Matrix size: {size}x{size}")
        
        # --- SVD ---
        print(f"\n  SVD:")
        
        # NumPy
        a_np = np.random.randn(size, size).astype(np.float32)
        numpy_svd_time = benchmark(lambda: np.linalg.svd(a_np), n_runs=5)
        print(f"    NumPy:         {numpy_svd_time['mean']:.2f} ± {numpy_svd_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', 'operation': 'SVD', **numpy_svd_time})
        
        # PyTorch CPU
        a_cpu = torch.from_numpy(a_np)
        torch_cpu_svd_time = benchmark(lambda: torch.linalg.svd(a_cpu), n_runs=5)
        print(f"    PyTorch (CPU): {torch_cpu_svd_time['mean']:.2f} ± {torch_cpu_svd_time['std']:.2f} ms "
              f"{format_speedup(numpy_svd_time['mean'], torch_cpu_svd_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', 'operation': 'SVD', **torch_cpu_svd_time})
        
        # PyTorch GPU
        if GPU:
            a_gpu = a_cpu.to(GPU)
            torch_gpu_svd_time = benchmark(lambda: torch.linalg.svd(a_gpu), n_runs=5)
            print(f"    PyTorch (GPU): {torch_gpu_svd_time['mean']:.2f} ± {torch_gpu_svd_time['std']:.2f} ms "
                  f"{format_speedup(numpy_svd_time['mean'], torch_gpu_svd_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', 'operation': 'SVD', **torch_gpu_svd_time})
        
        # --- Eigendecomposition (symmetric matrix) ---
        print(f"\n  Eigendecomposition (symmetric):")
        
        # Create symmetric matrix
        sym_np = (a_np + a_np.T) / 2
        
        # NumPy
        numpy_eig_time = benchmark(lambda: np.linalg.eigh(sym_np), n_runs=5)
        print(f"    NumPy:         {numpy_eig_time['mean']:.2f} ± {numpy_eig_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', 'operation': 'Eigh', **numpy_eig_time})
        
        # PyTorch CPU
        sym_cpu = torch.from_numpy(sym_np)
        torch_cpu_eig_time = benchmark(lambda: torch.linalg.eigh(sym_cpu), n_runs=5)
        print(f"    PyTorch (CPU): {torch_cpu_eig_time['mean']:.2f} ± {torch_cpu_eig_time['std']:.2f} ms "
              f"{format_speedup(numpy_eig_time['mean'], torch_cpu_eig_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', 'operation': 'Eigh', **torch_cpu_eig_time})
        
        # PyTorch GPU
        if GPU:
            sym_gpu = sym_cpu.to(GPU)
            torch_gpu_eig_time = benchmark(lambda: torch.linalg.eigh(sym_gpu), n_runs=5)
            print(f"    PyTorch (GPU): {torch_gpu_eig_time['mean']:.2f} ± {torch_gpu_eig_time['std']:.2f} ms "
                  f"{format_speedup(numpy_eig_time['mean'], torch_gpu_eig_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', 'operation': 'Eigh', **torch_gpu_eig_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 3: Linear Algebra (SVD, Eigendecomposition)")
print("="*60)
linalg_results = benchmark_linalg()

BENCHMARK 3: Linear Algebra (SVD, Eigendecomposition)

Matrix size: 256x256

  SVD:
    NumPy:         8.25 ± 0.24 ms
    PyTorch (CPU): 4.14 ± 0.02 ms 🚀 2.0x faster
    PyTorch (GPU): 6.09 ± 0.01 ms 🚀 1.4x faster

  Eigendecomposition (symmetric):
    NumPy:         4.09 ± 0.01 ms
    PyTorch (CPU): 1.39 ± 0.01 ms 🚀 2.9x faster
    PyTorch (GPU): 4.03 ± 0.03 ms 🚀 1.0x faster

Matrix size: 512x512

  SVD:
    NumPy:         43.58 ± 0.21 ms
    PyTorch (CPU): 15.64 ± 0.06 ms 🚀 2.8x faster
    PyTorch (GPU): 15.87 ± 0.03 ms 🚀 2.7x faster

  Eigendecomposition (symmetric):
    NumPy:         17.83 ± 0.02 ms
    PyTorch (CPU): 7.84 ± 2.65 ms 🚀 2.3x faster
    PyTorch (GPU): 11.13 ± 0.01 ms 🚀 1.6x faster

Matrix size: 1024x1024

  SVD:
    NumPy:         202.51 ± 1.28 ms
    PyTorch (CPU): 71.98 ± 0.14 ms 🚀 2.8x faster
    PyTorch (GPU): 48.99 ± 0.01 ms 🚀 4.1x faster

  Eigendecomposition (symmetric):
    NumPy:         75.98 ± 0.19 ms
    PyTorch (CPU): 20.14 ± 0.06 ms 🚀 3.8x faster
    Py

---

# Benchmark 4: Reductions (Sum, Mean, Std)

Aggregation operations over large arrays—essential for statistics and data analysis.

In [7]:
def benchmark_reductions(sizes: List[int] = [1_000_000, 10_000_000, 100_000_000]) -> pd.DataFrame:
    """Benchmark reduction operations: sum, mean, std."""
    results = []
    
    for size in sizes:
        print(f"\nArray size: {size:,} elements")
        
        # NumPy
        x_np = np.random.randn(size).astype(np.float32)
        numpy_time = benchmark(lambda: (np.sum(x_np), np.mean(x_np), np.std(x_np)))
        print(f"  NumPy:         {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', **numpy_time})
        
        # PyTorch CPU
        x_cpu = torch.from_numpy(x_np)
        torch_cpu_time = benchmark(lambda: (torch.sum(x_cpu), torch.mean(x_cpu), torch.std(x_cpu)))
        print(f"  PyTorch (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', **torch_cpu_time})
        
        # PyTorch GPU
        if GPU:
            x_gpu = x_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: (torch.sum(x_gpu), torch.mean(x_gpu), torch.std(x_gpu)))
            print(f"  PyTorch (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 4: Reductions (sum, mean, std)")
print("="*60)
reduction_results = benchmark_reductions()

BENCHMARK 4: Reductions (sum, mean, std)

Array size: 1,000,000 elements
  NumPy:         0.67 ± 0.01 ms
  PyTorch (CPU): 0.10 ± 0.00 ms 🚀 6.8x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 27.6x faster

Array size: 10,000,000 elements
  NumPy:         8.84 ± 0.53 ms
  PyTorch (CPU): 0.53 ± 0.01 ms 🚀 16.6x faster
  PyTorch (GPU): 0.04 ± 0.00 ms 🚀 223.3x faster

Array size: 100,000,000 elements
  NumPy:         89.82 ± 0.45 ms
  PyTorch (CPU): 24.90 ± 0.12 ms 🚀 3.6x faster
  PyTorch (GPU): 0.75 ± 0.00 ms 🚀 120.2x faster


---

# Benchmark 5: FFT (Fast Fourier Transform)

FFT is fundamental for signal processing, spectral analysis, and solving PDEs.

In [8]:
def benchmark_fft(sizes: List[int] = [2**16, 2**18, 2**20, 2**22]) -> pd.DataFrame:
    """Benchmark 1D and 2D FFT operations."""
    results = []
    
    print("\n--- 1D FFT ---")
    for size in sizes:
        print(f"\nSignal length: {size:,}")
        
        # NumPy
        x_np = np.random.randn(size).astype(np.float32)
        numpy_time = benchmark(lambda: np.fft.fft(x_np))
        print(f"  NumPy:         {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', 'operation': 'FFT1D', **numpy_time})
        
        # PyTorch CPU
        x_cpu = torch.from_numpy(x_np)
        torch_cpu_time = benchmark(lambda: torch.fft.fft(x_cpu))
        print(f"  PyTorch (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', 'operation': 'FFT1D', **torch_cpu_time})
        
        # PyTorch GPU
        if GPU:
            x_gpu = x_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: torch.fft.fft(x_gpu))
            print(f"  PyTorch (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', 'operation': 'FFT1D', **torch_gpu_time})
    
    print("\n--- 2D FFT ---")
    sizes_2d = [512, 1024, 2048, 4096]
    for size in sizes_2d:
        print(f"\nImage size: {size}x{size}")
        
        # NumPy
        x_np = np.random.randn(size, size).astype(np.float32)
        numpy_time = benchmark(lambda: np.fft.fft2(x_np))
        print(f"  NumPy:         {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', 'operation': 'FFT2D', **numpy_time})
        
        # PyTorch CPU
        x_cpu = torch.from_numpy(x_np)
        torch_cpu_time = benchmark(lambda: torch.fft.fft2(x_cpu))
        print(f"  PyTorch (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', 'operation': 'FFT2D', **torch_cpu_time})
        
        # PyTorch GPU
        if GPU:
            x_gpu = x_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: torch.fft.fft2(x_gpu))
            print(f"  PyTorch (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', 'operation': 'FFT2D', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 5: Fast Fourier Transform")
print("="*60)
fft_results = benchmark_fft()

BENCHMARK 5: Fast Fourier Transform

--- 1D FFT ---

Signal length: 65,536
  NumPy:         0.49 ± 0.00 ms
  PyTorch (CPU): 0.23 ± 0.02 ms 🚀 2.1x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 24.5x faster

Signal length: 262,144
  NumPy:         2.07 ± 0.00 ms
  PyTorch (CPU): 0.51 ± 0.00 ms 🚀 4.0x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 89.1x faster

Signal length: 1,048,576
  NumPy:         16.10 ± 0.18 ms
  PyTorch (CPU): 0.93 ± 0.01 ms 🚀 17.3x faster
  PyTorch (GPU): 0.03 ± 0.00 ms 🚀 598.6x faster

Signal length: 4,194,304
  NumPy:         88.73 ± 4.44 ms
  PyTorch (CPU): 18.95 ± 0.33 ms 🚀 4.7x faster
  PyTorch (GPU): 0.05 ± 0.00 ms 🚀 1743.2x faster

--- 2D FFT ---

Image size: 512x512
  NumPy:         2.44 ± 0.41 ms
  PyTorch (CPU): 0.16 ± 0.00 ms 🚀 15.1x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 160.2x faster

Image size: 1024x1024
  NumPy:         12.67 ± 0.09 ms
  PyTorch (CPU): 0.42 ± 0.01 ms 🚀 30.4x faster
  PyTorch (GPU): 0.02 ± 0.00 ms 🚀 591.1x faster

Image size: 2048x2048
  Nu

---

# Benchmark 6: Real-World Simulation — Heat Equation (PDE Solver)

This is where the PyTorch advantage becomes compelling. Let's solve the 2D heat equation using finite differences—a common task in scientific computing.

The key insight: **the exact same code runs on CPU or GPU with a single `.to(device)` call.**

In [9]:
def heat_equation_numpy(u: np.ndarray, alpha: float, dx: float, dt: float, steps: int) -> np.ndarray:
    """
    Solve 2D heat equation using finite differences.
    ∂u/∂t = α∇²u
    """
    factor = alpha * dt / (dx ** 2)
    
    for _ in range(steps):
        # Laplacian using finite differences
        laplacian = (
            np.roll(u, 1, axis=0) + np.roll(u, -1, axis=0) +
            np.roll(u, 1, axis=1) + np.roll(u, -1, axis=1) - 4 * u
        )
        u = u + factor * laplacian
    
    return u


def heat_equation_pytorch(u: torch.Tensor, alpha: float, dx: float, dt: float, steps: int) -> torch.Tensor:
    """
    Solve 2D heat equation using finite differences.
    SAME LOGIC as NumPy version — but runs on ANY device.
    """
    factor = alpha * dt / (dx ** 2)
    
    for _ in range(steps):
        # Laplacian using finite differences
        laplacian = (
            torch.roll(u, 1, dims=0) + torch.roll(u, -1, dims=0) +
            torch.roll(u, 1, dims=1) + torch.roll(u, -1, dims=1) - 4 * u
        )
        u = u + factor * laplacian
    
    return u


def benchmark_heat_equation(grid_sizes: List[int] = [256, 512, 1024, 2048], 
                            n_steps: int = 100) -> pd.DataFrame:
    """Benchmark the heat equation solver."""
    results = []
    alpha = 0.1  # Thermal diffusivity
    dx = 0.01    # Grid spacing
    dt = 0.0001  # Time step
    
    for size in grid_sizes:
        print(f"\nGrid size: {size}x{size}, {n_steps} time steps")
        
        # Initial condition: hot spot in center
        u_np = np.zeros((size, size), dtype=np.float32)
        center = size // 2
        u_np[center-10:center+10, center-10:center+10] = 100.0
        
        # NumPy
        numpy_time = benchmark(lambda: heat_equation_numpy(u_np.copy(), alpha, dx, dt, n_steps), n_runs=5)
        print(f"  NumPy:         {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'library': 'NumPy', 'device': 'CPU', **numpy_time})
        
        # PyTorch CPU
        u_cpu = torch.from_numpy(u_np)
        torch_cpu_time = benchmark(lambda: heat_equation_pytorch(u_cpu.clone(), alpha, dx, dt, n_steps), n_runs=5)
        print(f"  PyTorch (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'library': 'PyTorch', 'device': 'CPU', **torch_cpu_time})
        
        # PyTorch GPU — SAME CODE, just .to(GPU)
        if GPU:
            u_gpu = u_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: heat_equation_pytorch(u_gpu.clone(), alpha, dx, dt, n_steps), n_runs=5)
            print(f"  PyTorch (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'library': 'PyTorch', 'device': 'GPU', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 6: Heat Equation PDE Solver (Real-World Simulation)")
print("="*60)
heat_results = benchmark_heat_equation()

BENCHMARK 6: Heat Equation PDE Solver (Real-World Simulation)

Grid size: 256x256, 100 time steps
  NumPy:         6.08 ± 0.39 ms
  PyTorch (CPU): 7.51 ± 0.04 ms 🐢 1.2x slower
  PyTorch (GPU): 3.11 ± 0.01 ms 🚀 2.0x faster

Grid size: 512x512, 100 time steps
  NumPy:         19.65 ± 0.02 ms
  PyTorch (CPU): 8.45 ± 0.23 ms 🚀 2.3x faster
  PyTorch (GPU): 3.25 ± 0.05 ms 🚀 6.0x faster

Grid size: 1024x1024, 100 time steps
  NumPy:         73.21 ± 3.31 ms
  PyTorch (CPU): 14.80 ± 0.99 ms 🚀 4.9x faster
  PyTorch (GPU): 4.11 ± 0.00 ms 🚀 17.8x faster

Grid size: 2048x2048, 100 time steps
  NumPy:         815.62 ± 6.00 ms
  PyTorch (CPU): 370.80 ± 0.67 ms 🚀 2.2x faster
  PyTorch (GPU): 9.14 ± 0.04 ms 🚀 89.3x faster


---

# Benchmark 7: Batched Operations

PyTorch excels at batched computations—processing many samples simultaneously. This is common in Monte Carlo simulations, ensemble methods, and (of course) ML.

In [10]:
def benchmark_batched_matmul(batch_sizes: List[int] = [64, 256, 1024], 
                              matrix_size: int = 256) -> pd.DataFrame:
    """Benchmark batched matrix multiplication."""
    results = []
    
    for batch_size in batch_sizes:
        print(f"\nBatch size: {batch_size}, Matrix size: {matrix_size}x{matrix_size}")
        
        # NumPy — have to use einsum or loop
        a_np = np.random.randn(batch_size, matrix_size, matrix_size).astype(np.float32)
        b_np = np.random.randn(batch_size, matrix_size, matrix_size).astype(np.float32)
        numpy_time = benchmark(lambda: np.einsum('bij,bjk->bik', a_np, b_np), n_runs=5)
        print(f"  NumPy (einsum):  {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'batch_size': batch_size, 'library': 'NumPy', 'device': 'CPU', **numpy_time})
        
        # PyTorch CPU — native batched matmul
        a_cpu = torch.from_numpy(a_np)
        b_cpu = torch.from_numpy(b_np)
        torch_cpu_time = benchmark(lambda: torch.bmm(a_cpu, b_cpu), n_runs=5)
        print(f"  PyTorch (CPU):   {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'batch_size': batch_size, 'library': 'PyTorch', 'device': 'CPU', **torch_cpu_time})
        
        # PyTorch GPU
        if GPU:
            a_gpu = a_cpu.to(GPU)
            b_gpu = b_cpu.to(GPU)
            torch_gpu_time = benchmark(lambda: torch.bmm(a_gpu, b_gpu), n_runs=5)
            print(f"  PyTorch (GPU):   {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'batch_size': batch_size, 'library': 'PyTorch', 'device': 'GPU', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 7: Batched Matrix Multiplication")
print("="*60)
batched_results = benchmark_batched_matmul()

BENCHMARK 7: Batched Matrix Multiplication

Batch size: 64, Matrix size: 256x256
  NumPy (einsum):  328.09 ± 0.11 ms
  PyTorch (CPU):   1.67 ± 0.00 ms 🚀 196.3x faster
  PyTorch (GPU):   0.06 ± 0.00 ms 🚀 5911.9x faster

Batch size: 256, Matrix size: 256x256
  NumPy (einsum):  1318.73 ± 2.27 ms
  PyTorch (CPU):   8.00 ± 0.16 ms 🚀 164.8x faster
  PyTorch (GPU):   0.21 ± 0.00 ms 🚀 6423.6x faster

Batch size: 1024, Matrix size: 256x256
  NumPy (einsum):  5261.58 ± 8.65 ms
  PyTorch (CPU):   39.07 ± 2.85 ms 🚀 134.7x faster
  PyTorch (GPU):   0.80 ± 0.03 ms 🚀 6540.9x faster


---

# Benchmark 8: Automatic Differentiation — The Hidden Superpower

This is where PyTorch fundamentally changes what's possible. You get **exact gradients for free** on any computation.

NumPy? You'd have to derive and implement gradients manually, or use finite differences (slow and numerically unstable).

In [11]:
def complex_function_numpy(x: np.ndarray) -> np.ndarray:
    """A complex function that would be tedious to differentiate by hand."""
    return np.sum(np.sin(x @ x.T) * np.exp(-np.abs(x).sum(axis=1, keepdims=True)))


def numerical_gradient_numpy(x: np.ndarray, eps: float = 1e-5) -> np.ndarray:
    """Compute gradient using finite differences — slow and approximate."""
    grad = np.zeros_like(x)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x_plus = x.copy()
            x_plus[i, j] += eps
            x_minus = x.copy()
            x_minus[i, j] -= eps
            grad[i, j] = (complex_function_numpy(x_plus) - complex_function_numpy(x_minus)) / (2 * eps)
    return grad


def complex_function_pytorch(x: torch.Tensor) -> torch.Tensor:
    """Same function in PyTorch."""
    return torch.sum(torch.sin(x @ x.T) * torch.exp(-torch.abs(x).sum(dim=1, keepdim=True)))


def benchmark_autograd(sizes: List[int] = [16, 32, 64, 128]) -> pd.DataFrame:
    """Benchmark gradient computation: NumPy finite differences vs PyTorch autograd."""
    results = []
    
    for size in sizes:
        print(f"\nMatrix size: {size}x{size} ({size*size:,} gradient components)")
        
        # NumPy finite differences
        x_np = np.random.randn(size, size).astype(np.float32)
        numpy_time = benchmark(lambda: numerical_gradient_numpy(x_np), n_runs=3, n_warmup=1)
        print(f"  NumPy (finite diff): {numpy_time['mean']:.2f} ± {numpy_time['std']:.2f} ms")
        results.append({'size': size, 'method': 'NumPy finite diff', 'device': 'CPU', **numpy_time})
        
        # PyTorch autograd CPU
        x_cpu = torch.from_numpy(x_np).requires_grad_(True)
        def pytorch_grad_cpu():
            x_cpu.grad = None
            y = complex_function_pytorch(x_cpu)
            y.backward()
            return x_cpu.grad
        torch_cpu_time = benchmark(pytorch_grad_cpu, n_runs=10)
        print(f"  PyTorch autograd (CPU): {torch_cpu_time['mean']:.2f} ± {torch_cpu_time['std']:.2f} ms "
              f"{format_speedup(numpy_time['mean'], torch_cpu_time['mean'])}")
        results.append({'size': size, 'method': 'PyTorch autograd', 'device': 'CPU', **torch_cpu_time})
        
        # PyTorch autograd GPU
        if GPU:
            x_gpu = torch.from_numpy(x_np).to(GPU).requires_grad_(True)
            def pytorch_grad_gpu():
                x_gpu.grad = None
                y = complex_function_pytorch(x_gpu)
                y.backward()
                return x_gpu.grad
            torch_gpu_time = benchmark(pytorch_grad_gpu, n_runs=10)
            print(f"  PyTorch autograd (GPU): {torch_gpu_time['mean']:.2f} ± {torch_gpu_time['std']:.2f} ms "
                  f"{format_speedup(numpy_time['mean'], torch_gpu_time['mean'])}")
            results.append({'size': size, 'method': 'PyTorch autograd', 'device': 'GPU', **torch_gpu_time})
    
    return pd.DataFrame(results)

print("="*60)
print("BENCHMARK 8: Gradient Computation")
print("NumPy: Finite differences (slow, approximate)")
print("PyTorch: Automatic differentiation (fast, exact)")
print("="*60)
autograd_results = benchmark_autograd()

BENCHMARK 8: Gradient Computation
NumPy: Finite differences (slow, approximate)
PyTorch: Automatic differentiation (fast, exact)

Matrix size: 16x16 (256 gradient components)
  NumPy (finite diff): 3.20 ± 0.02 ms
  PyTorch autograd (CPU): 0.06 ± 0.00 ms 🚀 55.7x faster
  PyTorch autograd (GPU): 0.14 ± 0.01 ms 🚀 22.7x faster

Matrix size: 32x32 (1,024 gradient components)
  NumPy (finite diff): 17.23 ± 0.05 ms
  PyTorch autograd (CPU): 0.08 ± 0.01 ms 🚀 210.1x faster
  PyTorch autograd (GPU): 0.17 ± 0.00 ms 🚀 101.8x faster

Matrix size: 64x64 (4,096 gradient components)
  NumPy (finite diff): 90.71 ± 0.21 ms
  PyTorch autograd (CPU): 0.09 ± 0.01 ms 🚀 1026.5x faster
  PyTorch autograd (GPU): 0.14 ± 0.00 ms 🚀 656.7x faster

Matrix size: 128x128 (16,384 gradient components)
  NumPy (finite diff): 1125.49 ± 0.32 ms
  PyTorch autograd (CPU): 0.15 ± 0.01 ms 🚀 7632.9x faster
  PyTorch autograd (GPU): 0.15 ± 0.00 ms 🚀 7447.9x faster


---

# Summary: The PyTorch Advantage

In [12]:
print("\n" + "="*70)
print("SUMMARY: Why PyTorch for General Numerical Computing?")
print("="*70)

summary = """
┌─────────────────────────────────────────────────────────────────────┐
│  WRITE ONCE, ACCELERATE ANYWHERE                                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  The same PyTorch code runs on:                                     │
│    • CPU (your laptop)                                              │
│    • NVIDIA GPU (CUDA)                                              │
│    • Apple Silicon (MPS)                                            │
│    • AMD GPU (ROCm)                                                 │
│    • Intel GPU (XPU)                                                │
│    • Google TPU                                                     │
│    • Future accelerators (as PyTorch adds support)                  │
│                                                                     │
│  NumPy code? CPU only. Forever.                                     │
│                                                                     │
├─────────────────────────────────────────────────────────────────────┤
│  AUTOMATIC DIFFERENTIATION                                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  PyTorch: y.backward() → exact gradients, any computation           │
│  NumPy:   Manual derivation or slow finite differences              │
│                                                                     │
│  This enables:                                                      │
│    • Optimization problems (gradient descent for free)              │
│    • Sensitivity analysis                                           │
│    • Physics-informed computing                                     │
│    • Inverse problems                                               │
│                                                                     │
├─────────────────────────────────────────────────────────────────────┤
│  FUTURE-PROOF                                                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  PyTorch is actively developed with massive resources:              │
│    • Better kernels every release                                   │
│    • New hardware support                                           │
│    • torch.compile() for automatic optimization                     │
│    • Quantization, sparsity, mixed precision                        │
│                                                                     │
│  Your code today will be faster tomorrow — automatically.           │
│                                                                     │
├─────────────────────────────────────────────────────────────────────┤
│  ML-READY                                                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  When you need to add ML to your pipeline:                          │
│    • Data already in tensors                                        │
│    • Same device (no CPU↔GPU transfers)                             │
│    • Seamless integration with models                               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
"""
print(summary)


SUMMARY: Why PyTorch for General Numerical Computing?

┌─────────────────────────────────────────────────────────────────────┐
│  WRITE ONCE, ACCELERATE ANYWHERE                                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  The same PyTorch code runs on:                                     │
│    • CPU (your laptop)                                              │
│    • NVIDIA GPU (CUDA)                                              │
│    • Apple Silicon (MPS)                                            │
│    • AMD GPU (ROCm)                                                 │
│    • Intel GPU (XPU)                                                │
│    • Google TPU                                                     │
│    • Future accelerators (as PyTorch adds support)                  │
│                                                                     │
│  NumPy

---

# Bonus: torch.compile() — The Future is Now

PyTorch 2.0+ includes `torch.compile()`, which can automatically optimize your code with minimal changes.

In [13]:
# Check if torch.compile is available (PyTorch 2.0+)
if hasattr(torch, 'compile'):
    print("torch.compile() is available!")
    
    def demo_function(x: torch.Tensor) -> torch.Tensor:
        """A function with multiple operations that can be fused."""
        x = torch.sin(x)
        x = x * 2
        x = torch.relu(x)
        x = x + 1
        return x.sum()
    
    # Compile the function
    compiled_function = torch.compile(demo_function)
    
    size = 10_000_000
    x = torch.randn(size)
    
    print(f"\nBenchmarking on {size:,} elements:")
    
    # Eager mode
    eager_time = benchmark(lambda: demo_function(x))
    print(f"  Eager mode:    {eager_time['mean']:.2f} ms")
    
    # Compiled mode (first run includes compilation)
    compiled_time = benchmark(lambda: compiled_function(x), n_warmup=5)  # Extra warmup for compilation
    print(f"  Compiled mode: {compiled_time['mean']:.2f} ms {format_speedup(eager_time['mean'], compiled_time['mean'])}")
    
    if GPU:
        x_gpu = x.to(GPU)
        compiled_function_gpu = torch.compile(demo_function)
        
        eager_gpu_time = benchmark(lambda: demo_function(x_gpu))
        print(f"  Eager (GPU):   {eager_gpu_time['mean']:.2f} ms")
        
        compiled_gpu_time = benchmark(lambda: compiled_function_gpu(x_gpu), n_warmup=5)
        print(f"  Compiled (GPU): {compiled_gpu_time['mean']:.2f} ms {format_speedup(eager_gpu_time['mean'], compiled_gpu_time['mean'])}")
else:
    print("torch.compile() requires PyTorch 2.0+")
    print(f"Current version: {torch.__version__}")

torch.compile() is available!

Benchmarking on 10,000,000 elements:
  Eager mode:    12.17 ms
  Compiled mode: 0.32 ms 🚀 37.8x faster
  Eager (GPU):   0.06 ms
  Compiled (GPU): 0.03 ms 🚀 2.4x faster


---

# Conclusion

PyTorch isn't just for machine learning anymore. It's a **general-purpose numerical computing library** that happens to also be the best ML framework.

By writing your numerical code in PyTorch today, you get:

1. **Immediate GPU acceleration** — just add `.to('cuda')`
2. **Automatic differentiation** — gradients for any computation
3. **Future performance gains** — as PyTorch improves, your code gets faster
4. **ML integration** — when you need it, your data is ready

The question isn't "should I learn PyTorch for ML?" — it's "why am I still using NumPy for heavy computation?"

---

*Dávid Isztl | isztld.com*